참고 : https://www.data.go.kr/data/15084084/openapi.do

In [1]:
from modules.__init__ import *

USER_AGENT environment variable not set, consider setting it to identify your requests.



'''
category: 예보 항목
- TMN : 최저 기온 - 오전 6시
- TMX : 최고 기온 - 오후 3시
- SKY : 하늘 상태
- PTY : 강수 형태
'''

In [7]:
import pandas as pd
from datetime import datetime, timedelta
import pytz
from tqdm import tqdm
load_dotenv()
SERVICE_KEY = os.environ.get("WEATHER_DECODING_API_KEY")

In [8]:
position_df = pd.read_excel("position.xlsx")
position_df = position_df[position_df['2단계'].isna()][["1단계","격자 X","격자 Y"]]
position_df = position_df[position_df["1단계"].str.contains("특별시|광역시|경기도|제주|강원")]

In [64]:
class WeatherAPI:
    def __init__(self, 
                 service_key:str,
                 position_df:pd.DataFrame,
                 predict_type:str="단기예보조회"):
        self.service_key = service_key
        self.predict_type_dict = {
            "초단기실황조회":"getUltraSrtNcst",
            "초단기예보조회":"getUltraSrtFcst",
            "단기예보조회":"getVilageFcst",
            "예보버전조회":"getFcstVersion",
        }
        self.endpoinrt_url = f"http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/{self.predict_type_dict[predict_type]}"
        print("endpoinrt_url :",self.endpoinrt_url)
        self.request_params = {
            "serviceKey":self.service_key,
            "dataType":"JSON", # default 값 : XML
            "numOfRows":'10',
            "pageNo":'3',
        }
        self.position_df = position_df
        self._set_date_params()
        
    def _set_date_params(self):
        korea_timezone = pytz.timezone('Asia/Seoul')
        current_time = datetime.now(korea_timezone)+timedelta(days=-1)
        self.formatted_date = current_time.strftime('%Y%m%d')
        self.request_params["base_date"] = self.formatted_date
        
        
    def get_weather_data(self, 
                         category:str, 
                         fcst_time:str):
        
        # 이병신같은새끼 0600 값이 디폴트라해놓고, 날씨정보없을때가 많네 개병신같은거
        self.request_params["base_time"] = fcst_time
        for _, row in tqdm(self.position_df.iterrows()):
            self.request_params["nx"] = row["격자 X"]
            self.request_params["ny"] = row["격자 Y"]
            print(self.request_params)
            try:
                response = requests.get(self.endpoinrt_url, params=self.request_params)
                data = response.json()
                print(data)
                items = data['response']['body']['items']['item']
                found = next(filter(lambda x: x['category'] == category and x['fcstTime'] == fcst_time, items), None)

                if (found):
                    return found['fcstValue']
                else:
                    return None
            except Exception as e:
                print(f"에러발생 : {e}")
                return None

In [65]:
weather_api = WeatherAPI(service_key=SERVICE_KEY, 
                         position_df=position_df)
sky = weather_api.get_weather_data("TMP", "0700")
# precipitation = weather_api.get_weather_data("PTY", "0500")
# lowest_temp_of_today = weather_api.get_weather_data("TMN", "0600")
# highest_temp_of_today = weather_api.get_weather_data("TMX", "1500")

endpoinrt_url : http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst


0it [00:00, ?it/s]

{'serviceKey': '9hl2k5vn0vx2isYGtOWt05rZeKDnzUszpaI8sdVFYmHUsOUVg0gi9YRhcc7E9U3f8U5UMrxb/yiZfYu50vT8sg==', 'dataType': 'JSON', 'numOfRows': '10', 'pageNo': '3', 'base_date': '20250110', 'nx': 60, 'ny': 127}
{'response': {'header': {'resultCode': '01', 'resultMsg': 'APPLICATION_ERROR'}}}
에러발생 : 'body'


In [34]:
# import os

# from dotenv import load_dotenv

# load_dotenv()

# SERVICE_KEY = os.environ.get("WEATHER_DECODING_API_KEY")

# api_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"

# # 관측 위치 : 서울특별시 강남구 개포2동
# # 당일 새벽 2시에 발표된 예보를 기준으로 최저기온, 최고기온 정보를 가져옵니다.
# params = {
#     'serviceKey': SERVICE_KEY,
#     'numOfRows': '10',
#     'dataType': 'JSON',
#     'base_time': '0200',
#     'nx': '62',
#     'ny': '125',
# }

# '''
# category: 예보 항목
# - TMN : 최저 기온 - 오전 6시
# - TMX : 최고 기온 - 오후 3시
# - SKY : 하늘 상태
# - PTY : 강수 형태
# '''
# def fetch_data_from_kma(current_time_kst, page_no, category, fcst_time):
#     try:
#         date_format = "YYYYMMDD"
#         base_date = current_time_kst.format(date_format)
#         print("base_date :",base_date)
#         params['base_date'] = base_date
#         params['pageNo'] = page_no
#         print("params :",params)
#         response = requests.get(api_url, params=params)
#         print("response :",response)
#         response.raise_for_status()
#         data = response.json()

#         items = data['response']['body']['items']['item']
#         found = next(filter(lambda x: x['category'] == category and x['fcstTime'] == fcst_time, items), None)

#         if (found):
#             return found['fcstValue']
#         else:
#             return None

#     except requests.exceptions.HTTPError as errh:
#         print(f"HTTP Error: {errh}")
#     except requests.exceptions.ConnectionError as errc:
#         print(f"Error Connecting: {errc}")
#     except requests.exceptions.Timeout as errt:
#         print(f"Timeout Error: {errt}")
#     except requests.exceptions.RequestException as err:
#         print(f"Something went wrong: {err}")

#     return None

# # main.py

# import arrow
# current_time_kst = arrow.now('Asia/Seoul')
# date_format = "YYYY년 MM월 DD일 dddd"
# date_of_today = current_time_kst.format(date_format, locale="ko_kr")

# # 날씨 정보
# weather_msg = ""
# sky = fetch_data_from_kma(current_time_kst, '3', 'SKY', '0500')
# precipitation = fetch_data_from_kma(current_time_kst, '4', 'PTY', '0500')
# lowest_temp_of_today = fetch_data_from_kma(current_time_kst, '5', 'TMN', '0600')
# highest_temp_of_today = fetch_data_from_kma(current_time_kst, '16', 'TMX', '1500')

#     # if (sky == None or precipitation == None or
#     #     lowest_temp_of_today == None or highest_temp_of_today == None):
#     #     weather_msg = "날씨 정보를 가져오지 못했습니다. 😢"
#     # else:
#     #     weather_of_today = f"{STATUS_OF_SKY[sky]} (강수: {STATUS_OF_PRECIPITATION[precipitation]})"
#     #     weather_msg = (
#     #         f"🌏 현재 날씨: {weather_of_today}\n"
#     #         f"🔼 최고 기온: {highest_temp_of_today}°C\n"
#     #         f"🔽 최저 기온: {lowest_temp_of_today}°C\n"
#     #         f"🔎 관측 지점: 서울 강남구 개포2동"
#         # )

base_date : 20250111
params : {'serviceKey': '9hl2k5vn0vx2isYGtOWt05rZeKDnzUszpaI8sdVFYmHUsOUVg0gi9YRhcc7E9U3f8U5UMrxb/yiZfYu50vT8sg==', 'numOfRows': '10', 'dataType': 'JSON', 'base_time': '0200', 'nx': '62', 'ny': '125', 'base_date': '20250111', 'pageNo': '3'}
response : <Response [200]>
base_date : 20250111
params : {'serviceKey': '9hl2k5vn0vx2isYGtOWt05rZeKDnzUszpaI8sdVFYmHUsOUVg0gi9YRhcc7E9U3f8U5UMrxb/yiZfYu50vT8sg==', 'numOfRows': '10', 'dataType': 'JSON', 'base_time': '0200', 'nx': '62', 'ny': '125', 'base_date': '20250111', 'pageNo': '4'}
response : <Response [200]>
base_date : 20250111
params : {'serviceKey': '9hl2k5vn0vx2isYGtOWt05rZeKDnzUszpaI8sdVFYmHUsOUVg0gi9YRhcc7E9U3f8U5UMrxb/yiZfYu50vT8sg==', 'numOfRows': '10', 'dataType': 'JSON', 'base_time': '0200', 'nx': '62', 'ny': '125', 'base_date': '20250111', 'pageNo': '5'}
response : <Response [200]>
base_date : 20250111
params : {'serviceKey': '9hl2k5vn0vx2isYGtOWt05rZeKDnzUszpaI8sdVFYmHUsOUVg0gi9YRhcc7E9U3f8U5UMrxb/yiZfYu50vT8

In [35]:
current_time_kst

<Arrow [2025-01-11T01:21:25.790723+09:00]>

In [33]:
precipitation

'0'

In [29]:
lowest_temp_of_today

'-7.0'